In [1]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()

from src.data.Extract_segments_v3 import create_volumes, Console_and_file_logger
import logging, os, glob

Console_and_file_logger('extract_segments/heiner_test', logging.INFO)

%reload_ext autoreload
%autoreload 2

search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/3d-mri-domain-adaption


DEBUG:root:Creating directory /mnt/ssd/git/3d-mri-domain-adaption/logs/extract_segments
2020-10-13 08:51:44,493 INFO -------------------- Start --------------------
2020-10-13 08:51:44,493 INFO Working directory: /mnt/ssd/git/3d-mri-domain-adaption.
2020-10-13 08:51:44,493 INFO Log file: ./logs/extract_segments/heiner_test.log
2020-10-13 08:51:44,493 INFO Log level for console: INFO


# We need to check for typos or case sensitive differences in xml- and folder names

In [2]:
# Extract the masks and images from circle cvi42wsx export files
# works simultanous on all CPUs, adjust to your machine

# reads all xml file names
# reads all dicom file names
# maps xml and dicom images according to patient UID and date
# extracts the ll contour, contour types and series IDs from the xml files
# opens all dicoms with the given series-IDs from the mapped patient folder (we also need the dicom images between the labeled timesteps)
# sort/stack dicom images according to the Triggertime and Origin
# create a masked twin with the same shape but with the contours
# transform the contours to categorical masks (0 = Background, 1 = RV, 2 = Myocard, 3 = LV)
# copy all metadata from the original dicom images, + minor encoding cleaning
# builds three 4d volumes (MRI image, Mask, mixed Image)
# save 4d volumes as nrrd files to given export path
working_dir = os.getcwd()

# path to the folder, which contains the xml files
# path_to_xml = os.path.join(working_dir, '/mnt/data/datasets/cardio/new_gcn_extract/')
# path_to_xml = os.path.join(working_dir, 'data/interim/peters_cohort/')
path_to_xml = '/mnt/ssd/data/heiner_test_export/'

# Define the path to the dicom files
path_to_dicom = None # if None --> script expects the dicoms to be in a subfolder within the xml files
#path_to_dicom = '/mnt/data/datasets/cardio/ahf_export/'

# Define a folder where we can save the nrrd files to
path_to_export = os.path.join(working_dir, 'data/raw/heiner_test')

create_volumes(path_to_xml, path_to_dicom, path_to_export, max_workers=12, spawn_processes=True)

2020-10-13 08:52:44,522 INFO no dicom path given, expecting a dicom folder within the xml folder.
2020-10-13 08:52:44,523 INFO Found 1 xml-files.
2020-10-13 08:52:44,524 INFO Matched: 1 xml files to a dicom folder
2020-10-13 08:52:44,525 INFO Didn't find a dicom folder for: 0 xml files
2020-10-13 08:52:44,559 INFO --------------------------------------------------
2020-10-13 08:52:44,561 INFO processing patient: 1 (SAX_REXUUV3F200511121251461.3.46.670589.11.0.0.11.4.2.0.5167.5.2984.2005111212514448176) of total: 1
2020-10-13 08:52:45,321 INFO Image size: (256, 256, 1, 18)
2020-10-13 08:52:45,322 INFO Image Spacing: (1.3671875, 1.3671875, 8.80000019073486, 1)
2020-10-13 08:52:45,330 INFO process with patient 1 finished after 0.771 sec.
2020-10-13 08:52:45,338 INFO All 1 patients done in 0.813 s


# Ignore the following cells if you dont want to parse manually !!!

# Tests for the long axis contours and the manual labels - 

In [5]:
# get all xml files
import glob
xmls = glob.glob("data/temp/export/**/*.cvi42wsx")
xmls

['data/temp/export/0PTV75MP200506271208231/SAX_0PTV75MP200506271208231.3.46.670589.11.0.0.11.4.2.0.5167.5.5240.2005062712082160343.cvi42wsx',
 'data/temp/export/0PTV75MP200506271208231/LAX_0PTV75MP200506271208231.3.46.670589.11.0.0.11.4.2.0.5167.5.5240.2005062712082160343.cvi42wsx',
 'data/temp/export/0HQQW4ZN200705231338441/LAX_0HQQW4ZN200705231338441.2.124.113532.192.168.202.14.20070523.131344.1689214.cvi42wsx',
 'data/temp/export/0HQQW4ZN200705231338441/SAX_0HQQW4ZN200705231338441.2.124.113532.192.168.202.14.20070523.131344.1689214.cvi42wsx',
 'data/temp/export/0AE4R74L190001011238291/SAX_0AE4R74L190001011238291.3.12.2.1107.5.99.2.1013.30000008030612131778100072179.cvi42wsx',
 'data/temp/export/0AE4R74L190001011238291/LAX_0AE4R74L190001011238291.3.12.2.1107.5.99.2.1013.30000008030612131778100072179.cvi42wsx',
 'data/temp/export/04NEJQUZ200703130733401/LAX_04NEJQUZ200703130733401.3.46.670589.11.0.0.11.4.2.0.8365.5.7500.2007031307333781538.cvi42wsx',
 'data/temp/export/04NEJQUZ2007031

In [6]:
f_ = 'data/temp/data/04NEJQUZ_200703130733401/LAX_04NEJQUZ200703130733401.3.46.670589.11.0.0.11.4.2.0.8365.5.7500.2007031307333781538.cvi42wsx'
from bs4 import BeautifulSoup
# open xml file and create soup
with open(f_) as fp:
    soup = BeautifulSoup(fp, 'lxml')

In [7]:
all_items = list(soup.find_all('hash:item'))

In [8]:
cs = list()
for item in all_items:
    tag = item.attrs.get('hash:key', False)
    if tag:
        if '.' not in tag:
            cs.append(item.attrs.get('hash:key', False))

In [94]:
'lalaContour' in set(cs)

True

In [95]:
lala_c = []
lara_c = []

for item in all_items:
    if item.attrs.get('hash:key', False) == "lalaContour":
        lala_c.append(item)
    elif item.attrs.get('hash:key', False) == "laraContour":
        lara_c.append(item)

2019-11-14 21:49:52,528 INFO -------------------- Start --------------------
2019-11-14 21:49:52,529 INFO Working directory: /mnt/data/git/cardio.
2019-11-14 21:49:52,529 INFO Log file: ./logs/extract_segmentations_new_fast.log
2019-11-14 21:49:52,541 INFO -------------------- Start --------------------
2019-11-14 21:49:52,541 INFO Working directory: /mnt/data/git/cardio.
2019-11-14 21:49:52,541 INFO Log file: ./logs/extract_segmentations_new_fast.log


In [98]:
class Contour():

    def __init__(self, uid, image_size, pixel_size, points, sub_pixel_res, tag):
        self.uid = uid
        self.image_size = image_size
        self.pixel_size = pixel_size
        self.points = points
        self.sub_pixel_res = sub_pixel_res
        self.tag = tag

    uid = 0
    image_size = (0, 0)
    pixel_size = (0, 0)
    sub_pixel_res = 0
    points = []

def contour_factory(contours, tag):
    """
    Expects a list of segmentation-elements
    Reads/collects all neccessary data for each contour-element
    Wraps each contour information in a contour-object,
    builds a list of Contours and returns them
    Params: list of contour-xml tags:
    e.g. call:
    xml_file = 'D:\\git\\cardio\\data\\processed\\Segmentations\\0AE4R74L190001011238291.3.12.2.1107.5.99.2.1013.30000008030612131778100072179.cvi42wsx'
    with open(xml_file) as fp:
    soup = BeautifulSoup(fp, 'lxml')
    saendocardialContour = soup.find_all('hash:item', {'hash:key' : 'saendocardialContour'})

    """
    cont = {}
    logging.info('building {} contours'.format(len(contours)))
    for contour in contours:
        # print(contour)
        uid = contour.parent.parent.attrs.get('hash:key')
        image_width = contour.find('hash:item', {'hash:key': 'ImageSize'}).find('size:width').text
        image_height = contour.find('hash:item', {'hash:key': 'ImageSize'}).find('size:height').text
        pixel_width = contour.find('hash:item', {'hash:key': 'PixelSize'}).find('size:width').text
        pixel_height = contour.find('hash:item', {'hash:key': 'PixelSize'}).find('size:height').text
        sub_pixel = contour.find('hash:item', {'hash:key': 'SubpixelResolution'}).text
        points_t = contour.find('hash:item', {'hash:key': 'Points'})

        points_x = [int(int(point.text) / int(sub_pixel)) for point in points_t.find_all('point:x')]
        points_y = [int(int(point.text) / int(sub_pixel)) for point in points_t.find_all('point:y')]

        points = list(zip(points_x, points_y))

        # create a Contour class for each contour
        cont[str(uid)] = Contour(uid,
                                 (image_width, image_height),
                                 (pixel_width, pixel_height),
                                 points,
                                 sub_pixel, tag)
    return cont


"""
    Expects the Segmentation-XML-File as Beautifulsup-object
    Returns: a dictionary with segmentation-tag : list of segmentation-tag elements
    """
c = {}
contours = {'lalaContour': [],
            'laraContour': []}

all_items = list(soup.find_all('hash:item'))
lala_c = []
lara_c = []

for item in all_items:
    if item.attrs.get('hash:key', False) == "lalaContour":
        lala_c.append(item)
    elif item.attrs.get('hash:key', False) == "laraContour":
        lara_c.append(item)


contours['lalaContour'] = lala_c
contours['laraContour'] = lara_c

"""
convert to contour-objects
Expects a dictionary object with:
segmentation-tag : list of segmentation-tag elements
converts the list of segmentation-elements
into a flat dictionary with {uid : [contour]}
"""

for tag, contour_elements in contours.items():
    if len(contour_elements) > 0:
        logging.info('extract all {} segmentations'.format(tag))
        # transform all contour elements in contour objects
        contours = contour_factory(contour_elements, tag)
        # update contours dictionary
        # we might have multiple contours per slice
        # we have a list of contours with tuples of (uid, contour-obj)
        # we need to group all contours of one image with a list of contours
        for uid, cont in contours.items():
            # if this uid/image has already an
            if c.get(uid, False):
                c[uid].append(cont)
            else:
                c[uid] = [cont]

2019-11-14 21:54:43,610 INFO extract all lalaContour segmentations
2019-11-14 21:54:43,610 INFO building 2 contours
2019-11-14 21:54:43,644 INFO extract all laraContour segmentations
2019-11-14 21:54:43,645 INFO building 2 contours


In [99]:
c

{'1.3.46.670589.11.0.0.11.4.2.0.8365.5.2324.2007031307482373991': [<__main__.Contour at 0x7f3bc56dd748>,
 '1.3.46.670589.11.0.0.11.4.2.0.8365.5.2324.2007031307482321979': [<__main__.Contour at 0x7f3bc56dd7f0>,
  <__main__.Contour at 0x7f3bc56dda58>]}

In [ ]:
c = {}
contours = {'saendocardialContour': [],
        'saepicardialContour': [],
        'sarvendocardialContour': [],
        'sarvepicardialContour': []}

all_items = list(soup.find_all('hash:item'))
myo_c = []
rv_c = []
lv_c = []
for item in all_items:
    if item.attrs.get('hash:key', False) == "sarvendocardialContour":
        rv_c.append(item)
    elif item.attrs.get('hash:key', False) == "saendocardialContour":
        lv_c.append(item)
    elif item.attrs.get('hash:key', False) == "saepicardialContour":
        myo_c.append(item)

contours['sarvendocardialContour'] = rv_c
contours['saendocardialContour'] = lv_c
contours['saepicardialContour'] = myo_c